In [38]:
import pandas as pd 
import pysam
from xopen import xopen
from collections import defaultdict
import operator

In [49]:
df = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/atac/MAESTRO/test/20231121/A1008_2_MM_YSPB_WX_D_T7_EDTA5030/outs/cell_qc_metrics.tsv", sep='\t')

In [50]:
df = df.sort_values("overlap_peaks", ascending=False)

In [51]:
df

,barcode,fragments,fragments_overlapping_promoter,overlap_peaks,frac_peak,cell_called
11916,AACGCGGATCTTCCGATGGG,41648,17973,20026.0,0.4808,True
103880,AGCCCACTCATGCATCGGAG,31976,14382,16614.0,0.5196,True
197699,CAGACTGACACAGTTGGTCT,28152,13050,15226.0,0.5408,True
534265,TACGTCACTGTGGTTAGAGG,23552,11471,14723.0,0.6251,True
597182,TCGGAGGTCGGAATCAAGGA,25029,12352,14702.0,0.5874,True
...,...,...,...,...,...,...
194406,CAGACTACTCCGAATTCCCT,1,0,0.0,0.0000,False
449846,GGATAAGCTGTTCTAGTTAC,2,0,0.0,0.0000,False
194408,CAGACTACTCCGAATTGCGA,3,0,0.0,0.0000,False
449835,GGATAAGCTGTTCTACACAC,5,0,0.0,0.0000,False


In [53]:
df_new = pd.concat([df.iloc[:100], df.iloc[689527:]])

In [54]:
df_new

,barcode,fragments,fragments_overlapping_promoter,overlap_peaks,frac_peak,cell_called
11916,AACGCGGATCTTCCGATGGG,41648,17973,20026.0,0.4808,True
103880,AGCCCACTCATGCATCGGAG,31976,14382,16614.0,0.5196,True
197699,CAGACTGACACAGTTGGTCT,28152,13050,15226.0,0.5408,True
534265,TACGTCACTGTGGTTAGAGG,23552,11471,14723.0,0.6251,True
597182,TCGGAGGTCGGAATCAAGGA,25029,12352,14702.0,0.5874,True
...,...,...,...,...,...,...
194406,CAGACTACTCCGAATTCCCT,1,0,0.0,0.0000,False
449846,GGATAAGCTGTTCTAGTTAC,2,0,0.0,0.0000,False
194408,CAGACTACTCCGAATTGCGA,3,0,0.0,0.0000,False
449835,GGATAAGCTGTTCTACACAC,5,0,0.0,0.0000,False


In [55]:
target = set(df_new.barcode)

In [56]:
fq1 = "/SGRNJ07/Standard_Analysis/DATA05/limsdownload/23_10/2023_10_17/RD22110301/A1008_2_MM_YSPB_WX_D_T7_EDTA5030/2023-10-17-5/R231009018_R1.fastq.gz"
fq2 = "/SGRNJ07/Standard_Analysis/DATA05/limsdownload/23_10/2023_10_17/RD22110301/A1008_2_MM_YSPB_WX_D_T7_EDTA5030/2023-10-17-5/R231009018_R2.fastq.gz"
fq3 = "/SGRNJ07/Standard_Analysis/DATA05/limsdownload/23_10/2023_10_17/RD22110301/A1008_2_MM_YSPB_WX_D_T7_EDTA5030/2023-10-17-5/R231009018_R3.fastq.gz"
index = "/SGRNJ07/Standard_Analysis/DATA05/limsdownload/23_10/2023_10_17/RD22110301/A1008_2_MM_YSPB_WX_D_T7_EDTA5030/2023-10-17-5/R231009018_I1.fastq.gz"

In [57]:
fh_fq1 = xopen("/SGRNJ06/randd/USER/cjj/TESTDATA/maestro/rawdata/test_R1.fastq.gz", 'w')
fh_fq2 = xopen("/SGRNJ06/randd/USER/cjj/TESTDATA/maestro/rawdata/test_R2.fastq.gz", 'w')
fh_fq3 = xopen("/SGRNJ06/randd/USER/cjj/TESTDATA/maestro/rawdata/test_R3.fastq.gz", 'w')
fh_index1 = xopen("/SGRNJ06/randd/USER/cjj/TESTDATA/maestro/rawdata/test_I1.fastq.gz", 'w')

In [58]:
with pysam.FastxFile(fq1, persist=False) as fq1, \
    pysam.FastxFile(fq2, persist=False) as fq2, \
        pysam.FastxFile(fq3, persist=False) as fq3, \
            pysam.FastxFile(index, persist=False) as index1:

    for entry1, entry2, entry3, entry4 in zip(fq1, fq2, fq3, index1):
        bc = entry2.sequence[:6]+entry2.sequence[10:18]+entry2.sequence[22:28]
        if bc in target:
            fh_fq1.write(f"@{entry1.name}\n{entry1.sequence}\n+\n{entry1.quality}\n")
            fh_fq2.write(f"@{entry2.name}\n{entry2.sequence}\n+\n{entry2.quality}\n")
            fh_fq3.write(f"@{entry3.name}\n{entry3.sequence}\n+\n{entry3.quality}\n")
            fh_index1.write(f"@{entry4.name}\n{entry4.sequence}\n+\n{entry4.quality}\n")

In [59]:
fh_fq1.close()
fh_fq2.close()
fh_fq3.close()
fh_index1.close()

In [28]:
# convert 
bc_dict = dict()

In [29]:
whitelist_10X_fh = xopen("/SGRNJ06/randd/USER/cjj/Celescope/celeatac/CeleScope_ATAC/celescope/data/chemistry/atac/857K-2023.txt", 'r')

In [30]:
count_dict = defaultdict(int)

In [31]:
        with pysam.FastxFile("/SGRNJ06/randd/USER/cjj/TESTDATA/maestro/rawdata/test_R2.fastq.gz") as fq2_fh:
            for entry in fq2_fh:
                sgr_barcode = entry.sequence
                count_dict[sgr_barcode] += 1

In [32]:
count_dict = dict(sorted(count_dict.items(), key=operator.itemgetter(1), reverse=True))

In [33]:
for sgr_barcode in count_dict:
    bc_dict[sgr_barcode] = whitelist_10X_fh.readline().strip()

In [34]:
        out_fq1 = xopen("/SGRNJ06/randd/USER/cjj/TESTDATA/maestro/rawdata/test_R2_c.fastq.gz", 'w')

        with pysam.FastxFile("/SGRNJ06/randd/USER/cjj/TESTDATA/maestro/rawdata/test_R2.fastq.gz") as fq2_fh:
            for entry in fq2_fh:
                name = entry.name + ' ' + entry.comment
                new_seq = bc_dict[entry.sequence]
                new_qual = 'F' * len(new_seq)
                out_fq1.write(f'@{name}\n{new_seq}\n+\n{new_qual}\n')

        out_fq1.close()